In [1]:
import pandas as pd
import numpy as np

In [2]:
def replace_kvacice(string):
    replace_dict = {
        'š': 's',
        'č': 'c',
        'ć': 'c',
        'ž': 'z'
    }
    for kv, non_kv in replace_dict.items():
        string = string.replace(kv, non_kv)
    return string

strip_spaces = np.vectorize(lambda x: x.replace(' ', ''))
strip_kvacice = np.vectorize(replace_kvacice)
        
class FormResponse:
    def __init__(self, response: pd.DataFrame):
        self.response = response

    @classmethod
    def from_google_forms_csv(self, csv_path):
        """
        Inits a FormResponse object from a Google Forms csv export
        :param csv_path: path to the Google Forms csv export 
        """
        response_df = pd.read_csv(csv_path, index_col=False)
        return FormResponse(response_df)

    def get_meeting_data(self, 
                         scholar_column_name,
                         mentor_column_name,
                         meeting_column_name
                        ):
        
        response_df = self.response
        mapper = {meeting_column_name: 'meeting' }

        response_df['id'] = response_df[scholar_column_name] + response_df[mentor_column_name]
        response_df['id'] = strip_spaces(response_df['id'])
        response_df['id'] = strip_kvacice(response_df['id'])
        meeting_df = response_df[['id', meeting_column_name]].rename(columns=mapper).set_index('id')
        

        meeting_df.replace(np.nan, True, regex=True, inplace=True)
        meeting_df.replace(r'(.|\s)*\S(.|\s)*', False, regex=True, inplace=True)
        meeting_dict = meeting_df.to_dict()['meeting']
        return meeting_dict
        

In [3]:
class ReportSheet:
    
    def __init__(self, sheet):
        self.sheet = sheet
        
    @classmethod
    def from_google_sheets_csv(self, csv_path):
        sheet_df = pd.read_csv(csv_path)
        return ReportSheet(sheet_df)
    
    def meeting_report(self, meeting_dict):
        sheet = self.sheet
        sheet['id'] = self.sheet['Mentee'] + self.sheet['Mentor']
        sheet['id'] = strip_spaces(sheet['id'])
        sheet['id'] = strip_kvacice(sheet['id'])
        
        meet_column = []
        for meet_id in sheet.id:
            meeting = meeting_dict.get(meet_id)
            if meeting is None:
                meet_column.append('Unknown (report is missing)')
            elif meeting:
                meet_column.append('Active')
            else:
                meet_column.append('Inactive')
        sheet['Status - March 2022'] = meet_column
        sheet.drop(columns=['id'], inplace=True)
        return sheet
    
    def to_xlsx(self, xlsx_path):
        self.sheet.to_excel(xlsx_path, index=False, sheet_name='Tracking Juniors')

In [5]:
response = FormResponse.from_google_forms_csv(csv_path='../data/third_mentoring_report_junior_scholars.csv')
meeting_dict = response.get_meeting_data(scholar_column_name = 'First and Last Name',
                                         mentor_column_name = 'Mentor\'s First and Last ',
                                         meeting_column_name = 'If your answer in the previous question was less then or equal to 5, could you please shortly explain?'
                                         )
sheet = ReportSheet.from_google_sheets_csv('../data/sheet_junior_scholars.csv')
sheet.meeting_report(meeting_dict)
sheet.to_xlsx('../data/final_sheet.xlsx')
print('Done!')

Done!
